# Connecting and Uploading

In [ ]:
#loading Last Datasets:
import pandas as pd
dataset_5A = pd.read_pickle("Dataset (5 Authors).pkl")

In [ ]:
# taging:

dataset_5A['Merged'] = "<" + dataset_5A['Label_(number)'].astype(str) + "> " + dataset_5A['Sentence'] + " <end>"# Sentence Merge with their Label

display(dataset_5A.head(3))
print()

#Number of sample in each group:

num_samples = dataset_5A['Author'].value_counts()

display(num_samples)

print("\nSum = ", num_samples.sum())

,Sentence,Charles Dickens,Jane Austen,Mark Twain,Louisa May Alcott,Herman Melville,Author,Label_(number),Label_(tuple),Merged
0,"Why, I have been ashamed of your moroseness th...",1,0,0,0,0,Charles Dickens,0,"(1, 0, 0, 0, 0)","<0> Why, I have been ashamed of your morosenes..."
1,From the blacksmith’s.” “Halloa!” said the ser...,1,0,0,0,0,Charles Dickens,0,"(1, 0, 0, 0, 0)",<0> From the blacksmith’s.” “Halloa!” said the...
2,"Their conversations, however, were not always ...",0,1,0,0,0,Jane Austen,1,"(0, 1, 0, 0, 0)","<1> Their conversations, however, were not alw..."


,count
Author,
Charles Dickens,26856
Mark Twain,25527
Herman Melville,24011
Jane Austen,20044
Louisa May Alcott,19033



Sum =  115471


In [ ]:
# Example:
print(dataset_5A["Sentence"][10])

Gardeners are almost always pleasant, affable people to converse with; but beware of quarter-gunners, keepers of arsenals, and lonely light-house men.


# Library Installation

In [ ]:
#!pip install requests==2.31.0
!pip install requests==2.32.3

In [ ]:
#!pip install pyarrow==14.0.1
!pip install -U pyarrow==19.0.0

In [ ]:
!pip install datasets==2.10.0 #"datasets>=2.19,<3.0"

In [ ]:
!pip install transformers

In [ ]:
!pip install evaluate torch

In [ ]:
!pip install --upgrade evaluate

In [ ]:
!pip install accelerate

In [ ]:
!pip install numpy==2.0.2

In [ ]:
!pip install torchinfo

In [ ]:
!pip install wandb

# Generator (GPT-3)

## ---- Configuration:

In [ ]:
dataset_text = dataset_5A[['Merged']]

In [ ]:
dataset_text = dataset_text.rename(columns={'Merged': 'text'})

In [ ]:
# <0> = Charles Dickens
# <1> = Jane Austen
# <2> = Mark Twain
# <3> = Louisa May Alcott
# <4> = Herman Melville

In [ ]:
dataset_text

,text
0,"<0> Why, I have been ashamed of your morosenes..."
1,<0> From the blacksmith’s.” “Halloa!” said the...
2,"<1> Their conversations, however, were not alw..."
3,"<0> Thus, you were to hammer boys round—Old Cl..."
4,<0> “I am extremely sorry; but I knew there wa...
...,...
115466,<4> It was in vain to seek information on the ...
115467,"<4> At any rate, thought I, he will return wit..."
115468,"<4> Here we lay forty-eight hours, during whic..."
115469,<1> or is it equally criminal in every body? <...


In [ ]:
# Example:
dataset_text["text"][0]

'<0> Why, I have been ashamed of your moroseness there! <end>'

In [ ]:
len(dataset_text)

115471

In [ ]:
#pip install transformers

In [ ]:
#pip install evaluate torch

In [ ]:
#pip install --upgrade pyarrow evaluate

In [ ]:
#pip install accelerate

In [ ]:
#!pip install datasets==2.10.0

In [ ]:
import pyarrow as pa
print(pa.__version__)  #pip install "pyarrow<21"
print(hasattr(pa, "PyExtensionType"))

19.0.0
True


In [ ]:
import torch
import pandas as pd
from datasets import Dataset as HFDataset

In [ ]:
#pip install numpy==1.26.4

In [ ]:
from transformers import GPT2Tokenizer, GPTNeoForCausalLM, Trainer, TrainingArguments

In [ ]:
# Load pre-trained model and tokenizer
model_name = 'EleutherAI/gpt-neo-1.3B'
#model_name = 'EleutherAI/gpt-neo-125m'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

In [ ]:
# Add a padding token if it does not exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPTNeoForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50258, 2048)

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    encoding = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    encoding['labels'] = encoding['input_ids'].copy()
    return encoding

In [ ]:
import pandas as pd

# Convert pandas DataFrame to Hugging Face Dataset
hf_dataset_5A = HFDataset.from_pandas(dataset_text)

In [ ]:
hf_dataset_5A

Dataset({
    features: ['text'],
    num_rows: 115471
})

## ---- Training:

In [ ]:
# Apply the tokenize function to the dataset
tokenized_datasets = hf_dataset_5A.map(tokenize_function, batched=True)

Map:   0%|          | 0/115471 [00:00<?, ? examples/s]

In [ ]:
# Split dataset into train and eval
train_size = int(0.8 * len(tokenized_datasets))
train_dataset = tokenized_datasets.select(range(train_size))
eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))

In [ ]:
from datetime import datetime

base_dir = '/stylometry'
run_name             = "writing_style_v1"
current_date = datetime.now().strftime("%Y.%m.%d")

# Define save paths in Google Drive with date
drive_model_path     = f'{base_dir}/fine_tuned_model_gptNeo_1.3B/saved_model_{current_date}'
drive_tokenizer_path = f'{base_dir}/fine_tuned_model_gptNeo_1.3B/saved_tokenizer_{current_date}'
output_dir           = f'{base_dir}/fine_tuned_model_gptNeo_1.3B/results_{current_date}'
logging_dir          = f'{base_dir}/fine_tuned_model_gptNeo_1.3B/logs_{current_date}'

In [ ]:
print(run_name,":")
print(drive_model_path)
print(drive_tokenizer_path)
print(output_dir)
print(logging_dir)

In [ ]:
#pip install wandb

In [ ]:
import wandb

In [ ]:
#!WANDB_START_METHOD=thread

In [ ]:
#!WANDB_HTTP_TIMEOUT=300

In [ ]:
# Initialize the wandb session
wandb.init(project=run_name, entity="niu")

In [ ]:
!wandb login

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    run_name = run_name,               # name for run
    output_dir = output_dir,           # output directory
    num_train_epochs = 3,             # number of training epochs (10)
    per_device_train_batch_size = 16,  # batch size for training
    per_device_eval_batch_size = 16,   # batch size for evaluation
    warmup_steps = 1000,               # number of warmup steps for learning rate scheduler
    weight_decay = 0.01,               # strength of weight decay
    logging_dir = logging_dir,         # directory for storing logs
    report_to="wandb",                 # report metrics to wandb
    save_steps = 2000,                 # save checkpoint every 2000 steps
    save_total_limit = 10,             # keep only the last 5 checkpoints
    save_strategy = "steps",           # save based on steps (other option: "epoch")
)

In [ ]:
# Use Trainer to fine-tune the model
trainer = Trainer(
    model         = model,
    args          = training_args,
    train_dataset = train_dataset,
    eval_dataset  = eval_dataset,
)

In [ ]:
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from torchinfo import summary

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Tokenize a sample input and move to the same device
input_text = "Hello, this is a test input for model summary."
input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(device)

# Print the detailed summary
summary(model, input_data=input_ids)

Layer (type:depth-idx)                                  Output Shape              Param #
GPTNeoForCausalLM                                       [1, 16, 11, 128]          --
├─GPTNeoModel: 1-1                                      [1, 16, 11, 128]          --
│    └─Embedding: 2-1                                   [1, 11, 2048]             102,928,384
│    └─Embedding: 2-2                                   [1, 11, 2048]             4,194,304
│    └─Dropout: 2-3                                     [1, 11, 2048]             --
│    └─ModuleList: 2-4                                  --                        --
│    │    └─GPTNeoBlock: 3-1                            [1, 11, 2048]             50,352,128
│    │    └─GPTNeoBlock: 3-2                            [1, 11, 2048]             50,352,128
│    │    └─GPTNeoBlock: 3-3                            [1, 11, 2048]             50,352,128
│    │    └─GPTNeoBlock: 3-4                            [1, 11, 2048]             50,352,128
│    │    └─

In [ ]:
# Fine-tune the model
trainer.train()

Step,Training Loss
500,3.952100
1000,0.768300
1500,0.781200
2000,0.771200
2500,0.764600
3000,0.766600
3500,0.756400
4000,0.756500
4500,0.740100
5000,0.749700


TrainOutput(global_step=17322, training_loss=0.6543788440837801, metrics={'train_runtime': 19252.6907, 'train_samples_per_second': 14.394, 'train_steps_per_second': 0.9, 'total_flos': 2.5720071459648307e+17, 'train_loss': 0.6543788440837801, 'epoch': 3.0})

In [ ]:
# Saving ===================================================
model.save_pretrained(drive_model_path)
tokenizer.save_pretrained(drive_tokenizer_path)

print(f"Model saved to: {drive_model_path}")
print(f"Tokenizer saved to: {drive_tokenizer_path}")

In [ ]:
# Ensure the padding token is set if it was added
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

## ---- Loading:

In [ ]:
drive_model_path     = 'fine_tuned_model_gptNeo_1.3B/saved_model_2025.01.03'
drive_tokenizer_path = 'fine_tuned_model_gptNeo_1.3B/saved_tokenizer_2025.01.03'
output_dir           = 'fine_tuned_model_gptNeo_1.3B/results_2025.01.03'
logging_dir          = 'fine_tuned_model_gptNeo_1.3B/logs_2025.01.03'

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

# Load the model and tokenizer
model = GPTNeoForCausalLM.from_pretrained(drive_model_path)
tokenizer = GPT2Tokenizer.from_pretrained(drive_tokenizer_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Generate text
def generate_text(input_text, model, tokenizer, max_length=50, num_return_sequences=1):
    inputs = tokenizer.encode(input_text, return_tensors='pt')
    attention_mask = torch.ones(inputs.shape, dtype=torch.long)
    outputs = model.generate(inputs,
                             max_length=max_length,
                             num_return_sequences=num_return_sequences,
                             pad_token_id=tokenizer.eos_token_id,
                             attention_mask=attention_mask)
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts

In [ ]:
#Labels of Authors:

tages = [
         "<0> ",  # Charles Dickens
         "<1> ",  # Jane Austen
         "<2> ",  # Mark Twain
         "<3> ",  # Louisa May Alcott
         "<4> "   # Herman Melville
        ]

In [ ]:
# Example usage

start_with = "Hi"

for tag in tages:
    input_text = tag + start_with
    generated_texts = generate_text(input_text, model, tokenizer)
    print(generated_texts[0])

<0> Hi!” “Halloa, Pip!” said Joe, looking up from his work, and seeing me at the same time. <end>
<1> Hi!” cried Frank Churchill, “here’s a letter from Mrs. Weston to _me_. <end>
<2> Hi, Huck, you hear?” “Huck, it’s the watchman. <end>
<3> Hi, Jo!” “How are you, my darlings?” “All right, thanks. <end>
<4> Hi, there!--Here's a fine fellow!" <end>


In [ ]:
import random

grouped = dataset_5A.groupby('Label_(number)')

def sample_and_extract(group):
  sampled_sentences = group.sample(n=1000)  # Randomly select 1000 sentences form each author
  first_words = sampled_sentences['Sentence'].apply(lambda x: x.split()[0])  # Extract first word
  # Create a new DataFrame from the extracted words and labels
  result_df = pd.DataFrame({'First_Word': first_words, 'Label': group.name})
  return result_df

randome_start_words_gpt3 = grouped.apply(sample_and_extract, include_groups=False)
randome_start_words_gpt3.reset_index(inplace=True) #ungrouping
randome_start_words_gpt3.drop(['Label_(number)','level_1'], axis=1, inplace=True)

In [ ]:
randome_start_words_gpt3

,First_Word,Label
0,It,0
1,When,0
2,That’s,0
3,In,0
4,And,0
...,...,...
4995,and,4
4996,Neither,4
4997,"""We",4
4998,But,4


In [ ]:
import torch
from tqdm.notebook import tqdm
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# Build prompts robustly
prompts = [f"{tag}{s.split()[0]}"
           for s in randome_start_words_gpt3["First_Word"].astype(str)
           for tag in tages]

batch_size = 64        # you tested 64 OK on A100
max_new_tokens = 64

generated = []
with torch.inference_mode(), torch.amp.autocast("cuda", dtype=torch.float16):
    for i in tqdm(range(0, len(prompts), batch_size), desc="Generating (batched)", unit="batch"):
        batch = prompts[i:i+batch_size]
        enc = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)  # <<< move to CUDA
        out = model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
        texts = tokenizer.batch_decode(out, skip_special_tokens=True)
        generated.extend(texts)

generated_texts_gpt3 = pd.DataFrame({"Text": generated})


Generating (batched):   0%|          | 0/391 [00:00<?, ?batch/s]

In [ ]:
generated_texts_gpt3

,Text
0,<0> It is not my object to enlarge upon her gr...
1,<1> It must have been such a blow to her to lo...
2,<2> It’s the sickiest place I ever see. <end> ...
3,<3> It’s nothing but a fashionable hair-powder...
4,<4> It was a curious sight to see a ship thus ...
...,...
24995,"<0> At you, sir!” “Ah!” said the turnkey, glan..."
24996,"<1> At once, and without any preparation, she ..."
24997,<2> At once the chief mate said: “We have seen...
24998,"<3> At a sign from Demi, they all filed out in..."


In [ ]:
for text in generated_texts_gpt3["Text"].head(50):
    print(text)

<0> It is not my object to enlarge upon her grief, or allude to her reproach. <end>���3d3d�3d�3d3d3 PenguID
<1> It must have been such a blow to her to lose the man she had once thought so highly of! <end>;;
<2> It’s the sickiest place I ever see. <end> srf srf srf srf shenan srf
<3> It’s nothing but a fashionable hair-powder, and it doesn’t much matter as it’s not like the hair of any of your sex.” “Oh, it’s just perfectly detestable! <end>
<4> It was a curious sight to see a ship thus encircled by a company of savages, who, from their elevated station on the gun-deck, could not be supposed to have any thing to do with the concerns of the deck below. <end>
<0> When it came to this, I really sat down and wrote to him by the hour together, not knowing what else to do. <end>
<1> Whenever you are here, I wish you would order a carriage. <end> srf srf srf srfⓘ srf
<2> Whenhalf-past twelve rolled away, we heard the church clock strike twelve! <end>kHzkHz analgesVERTISEMENTnsicVERTISEMENT
<3

In [ ]:
# Save to CSV (without index column)
generated_texts_gpt3.to_csv("generated_texts_gpt3.csv", index=False)

#  Generator (GPT-3 + LoRA)

## ---- Configuration:

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPTNeoForCausalLM, Trainer, TrainingArguments
from datasets import Dataset as HFDataset
from datetime import datetime
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import pandas

In [ ]:
#Number of sample in each group:

num_samples = dataset_5A['Author'].value_counts()

display(num_samples)

print("\nSum = ", num_samples.sum())



,count
Author,
Charles Dickens,26856
Mark Twain,25527
Herman Melville,24011
Jane Austen,20044
Louisa May Alcott,19033



Sum =  115471


In [ ]:
# Prepare dataset for fine-tuning
dataset_text = dataset_5A[['Merged']].rename(columns={'Merged': 'text'})

In [ ]:
# Load pre-trained model and tokenizer
model_name = 'EleutherAI/gpt-neo-1.3B'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

In [ ]:
# Add a padding token if it does not exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPTNeoForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50258, 2048)

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    encoding = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    encoding['labels'] = encoding['input_ids'].copy()
    return encoding

In [ ]:
# Convert pandas DataFrame to Hugging Face Dataset
hf_dataset_5A = HFDataset.from_pandas(dataset_text)

In [ ]:
# Apply tokenization
tokenized_datasets = hf_dataset_5A.map(tokenize_function, batched=True)

Map:   0%|          | 0/115471 [00:00<?, ? examples/s]

## ---- LoRA Configuration:

In [ ]:
# Define LoRA configuration
lora_config = LoraConfig( task_type=TaskType.CAUSAL_LM,
                          r=8,  # Rank of the update matrices
                          lora_alpha=32,  # Alpha parameter for LoRA scaling
                          lora_dropout=0.1,  # Dropout probability for LoRA layers
                          bias="none",  # Don't train bias parameters
                          target_modules=["attn.attention.q_proj", "attn.attention.v_proj", "attn.attention.k_proj", "attn.attention.out_proj"]  # Target attention and MLP modules
                        )

In [ ]:
# Prepare the model with LoRA adapters
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
# Print trainable parameters info
model.print_trainable_parameters()

trainable params: 3,145,728 || all params: 1,318,723,584 || trainable%: 0.2385


## ---- Training:

In [ ]:
# Split dataset into train and eval
train_size = int(0.8 * len(tokenized_datasets))
train_dataset = tokenized_datasets.select(range(train_size))
eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))

In [ ]:
# Define paths and filenames
base_dir = 'stylometry' # Local directory for saving
run_name             = "writing_style_lora_v1"
current_date = datetime.now().strftime("%Y.%m.%d")

# Define save paths in Google Drive with date
lora_model_path     = f'{base_dir}/fine_tuned_model_gptNeo_1.3B_lora/saved_model_{current_date}'
drive_tokenizer_path = f'{base_dir}/fine_tuned_model_gptNeo_1.3B_lora/saved_tokenizer_{current_date}'
output_dir           = f'{base_dir}/fine_tuned_model_gptNeo_1.3B_lora/results_{current_date}'
logging_dir          = f'{base_dir}/fine_tuned_model_gptNeo_1.3B_lora/logs_{current_date}'


In [ ]:
print(run_name,":")
print(lora_model_path)
print(drive_tokenizer_path)
print(output_dir)
print(logging_dir)

In [ ]:
#pip install wandb

In [ ]:
import wandb

In [ ]:
#!WANDB_START_METHOD=thread

In [ ]:
#!WANDB_HTTP_TIMEOUT=300

In [ ]:
# Initialize the wandb session
wandb.init(project=run_name, entity="niu")

In [ ]:
!wandb login

wandb: Currently logged in as: rezaei-mosab (mosi-niu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:

# Define training arguments - reduced batch size for memory efficiency
training_args = TrainingArguments(
    run_name=run_name,
    output_dir=output_dir,
    num_train_epochs=3,  # Keep same as original
    per_device_train_batch_size=8,  # Reduced for memory efficiency
    per_device_eval_batch_size=8,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_dir=logging_dir,
    save_steps=2000,
    save_total_limit=5,
    save_strategy="steps",
    eval_strategy="steps",
    eval_steps=5000,
    fp16=False,
    report_to="none"
)

In [ ]:
# Set up trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,)

In [ ]:
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from torchinfo import summary

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Tokenize a sample input and move to the same device
input_text = "Hello, this is a test input for model summary."
input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(device)

# Print the detailed summary
summary(model, input_data=input_ids)

Layer (type:depth-idx)                                                 Output Shape              Param #
PeftModelForCausalLM                                                   --                        --
├─LoraModel: 1-1                                                       --                        --
│    └─GPTNeoForCausalLM: 2-1                                          --                        --
│    │    └─GPTNeoModel: 3-1                                           --                        1,318,723,584
│    │    └─Linear: 3-2                                                [1, 11, 50258]            (102,928,384)
Total params: 1,421,651,968
Trainable params: 3,145,728
Non-trainable params: 1,418,506,240
Total mult-adds (Units.GIGABYTES): 1.42
Input size (MB): 0.00
Forward/backward pass size (MB): 69.91
Params size (MB): 5686.61
Estimated Total Size (MB): 5756.52

In [ ]:
# Fine-tune the model
trainer.train()

Step,Training Loss,Validation Loss
5000,8.824300,8.895394
10000,8.836300,8.887632
15000,8.812400,8.884025
20000,8.827100,8.881921
25000,8.805700,8.880543
30000,8.793900,8.879627


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

TrainOutput(global_step=34641, training_loss=8.832486607750976, metrics={'train_runtime': 17584.1698, 'train_samples_per_second': 15.76, 'train_steps_per_second': 1.97, 'total_flos': 2.578702334259364e+17, 'train_loss': 8.832486607750976, 'epoch': 3.0})

In [ ]:
# Saving ===================================================
model.save_pretrained(lora_model_path)
tokenizer.save_pretrained(drive_tokenizer_path)

print(f"Model saved to: {lora_model_path}")
print(f"Tokenizer saved to: {drive_tokenizer_path}")

## ---- Loading:

In [ ]:
lora_model_path      = 'fine_tuned_model_gptNeo_1.3B_lora/saved_model_2025.08.09'
drive_tokenizer_path = 'fine_tuned_model_gptNeo_1.3B_lora/saved_tokenizer_2025.08.09'
output_dir           = 'fine_tuned_model_gptNeo_1.3B_lora/results_2025.08.09'
logging_dir          = 'fine_tuned_model_gptNeo_1.3B_lora/logs_2025.08.09'

In [ ]:
#output_dir           = ''
#logging_dir          = ''

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from peft import PeftModel, PeftConfig

# Load the model and tokenizer
peft_config = PeftConfig.from_pretrained(lora_model_path)
base_model = GPTNeoForCausalLM.from_pretrained(peft_config.base_model_name_or_path)
tokenizer = GPT2Tokenizer.from_pretrained(drive_tokenizer_path)
base_model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(base_model, lora_model_path)

In [ ]:
# Generate text
def generate_text(input_text, model, tokenizer, max_length=50, num_return_sequences=1):
    inputs = tokenizer.encode(input_text, return_tensors='pt')
    attention_mask = torch.ones(inputs.shape, dtype=torch.long)
    outputs = model.generate(inputs,
                             max_length=max_length,
                             num_return_sequences=num_return_sequences,
                             pad_token_id=tokenizer.eos_token_id,
                             attention_mask=attention_mask)
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts

In [ ]:
#Labels of Authors:

tages = [
         "<0> ",  # Charles Dickens
         "<1> ",  # Jane Austen
         "<2> ",  # Mark Twain
         "<3> ",  # Louisa May Alcott
         "<4> "   # Herman Melville
        ]

In [ ]:
# Example usage

start_with = "Hi"

for tag in tages:
    input_text = tag + start_with
    generated_texts = generate_text(input_text, model, tokenizer)
    print(generated_texts[0])

In [ ]:
# "First word sampleing" has been removed to have "randome_start_words_gpt3" same as GPT-3 FFT.

In [ ]:
randome_start_words_gpt3

In [ ]:
import torch
from tqdm.notebook import tqdm
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# Build prompts robustly
prompts = [f"{tag}{s.split()[0]}"
           for s in randome_start_words_gpt3["First_Word"].astype(str)
           for tag in tages]

batch_size = 64        # you tested 64 OK on A100
max_new_tokens = 64

generated = []
with torch.inference_mode(), torch.amp.autocast("cuda", dtype=torch.float16):
    for i in tqdm(range(0, len(prompts), batch_size), desc="Generating (batched)", unit="batch"):
        batch = prompts[i:i+batch_size]
        enc = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)  # <<< move to CUDA
        out = model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
        texts = tokenizer.batch_decode(out, skip_special_tokens=True)
        generated.extend(texts)

generated_texts_gpt3_lora = pd.DataFrame({"Text": generated})


Generating (batched):   0%|          | 0/391 [00:00<?, ?batch/s]

In [ ]:
generated_texts_gpt3_lora

,Text
0,<0> It’s quite a question.’ It was a question ...
1,"<1> It then had occurred to him, that, if Elin..."
2,"<2> It’s all very good. <end> ¥ minced (?, !ーテ..."
3,"<3> It’s good of her, you know,” said Amy, loo..."
4,<4> It was too late for me to do anything towa...
...,...
24995,"<0> Atterry, who had been in the company of th..."
24996,<1> Atfelice had now become so much attached t...
24997,<2> Atgrel went to the store to buy some money...
24998,"<3> Atgarts and ladders, all on the topmost st..."


In [ ]:
for text in generated_texts_gpt3_lora["Text"].head(50):
    print(text)

<0> It’s quite a question.’ It was a question whether my mother herself would have known how to give it a course of conduct which would have been the contrary of what I had done. <end> taxpiverpool Occupations – haircarned himself the favourite is something else entirely to note’’ The
<1> It then had occurred to him, that, if Elinor did not intend to return, he must know her very intentions. <end> Strongh _"—protect" —aunder your partner"— the _clout_ is very little", was the reply, that the hearty smile which accompanied her statement turned _her_ into
<2> It’s all very good. <end> ¥ minced (?, !ーティ&Bsolici^!) tbsp?" abackness's been gone through as the result again the moment his appearance was recalled from a long vacation ( tbsp!"< was the answer— a lie!) <basketball was the favourite of the boy— of
<3> It’s good of her, you know,” said Amy, looking round to see if there had been any “spoiling”. <end> cumberspeaceiveness is a sufficient to do him or his family well", again to prove

In [ ]:
# Save to CSV (without index column)
generated_texts_gpt3_lora.to_csv("generated_texts_gpt3_lora.csv", index=False)